<h1> ______________________________________ </h1>

<h1> Fiber barrel simulation analysis </h1>
<h1> ______________________________________ </h1>

<p style="font-size: 17px; color: black;"> In this Notebook we analyse the results from photons generated in 10000 MC events simulated in a Fiber barrel with photosensors in one end and Aluminium in the other end. </p>


<p style="font-size: 17px; color: black;"> For this, we first run some test to check that the MC output is indeed the type of events we wanted to simulate, later we calculate the system efficiency. </p>



<h1> ____________ </h1>
<h2> This version </h2>
<h1> ____________ </h1>

-  <p style="font-size: 17px; color: black;"> 10K events. </p>
-  <p style="font-size: 17px; color: black;"> Using perfect photosensors. </p>


-  <p style="font-size: 17px; color: black;"> Geant4 fundamental units: </p>

 -  <p style="font-size: 17px; color: black;"> Length [L]: mm (milimeter) </p>
 -  <p style="font-size: 17px; color: black;"> Time [T]: ns (nanosecond) </p>
 -  <p style="font-size: 17px; color: black;"> Energy [E]: MeV (megaelectronvolt) </p>



In [1]:
import numpy              as np
import pandas             as pd

In [2]:
import scipy              
from scipy                import stats, integrate
from scipy.signal         import convolve

In [3]:
import matplotlib.pyplot  as plt

from matplotlib.ticker    import FormatStrFormatter # to set the format of the plot's axis
from matplotlib.patches   import Rectangle # To add blanck spaces in tabular legends

In [4]:
import os
import math
import tables             as tb

<h1> _________________________________________________________________________________________________________ </h1>


<h1> __________________________ </h1>
<h2> Functions </h2>

-  <p style="font-size: 17px; color: black;"> <b> gauss(x, a,  mu, sigma): </b> Gaussian normalized to $a$. </p>


-  <p style="font-size: 17px; color: black;"> <b> gauss_sum(x, a0,  mu0, sigma0, a1, mu1, sigma1): </b> Sum of 2 gaussians. </p>


-  <p style="font-size: 17px; color: black;"> <b> dirac(x, x0): </b> Dirac delta at $x0$. </p>




In [5]:
# Fitting distributions to define

# note: pdf are normilized to 1 so we have yo add an amplitude param
def gauss(x, a,  mu, sigma):
    
    gaussian = stats.norm.pdf(np.sort(x), mu, sigma) 
    
    return (a/gaussian.sum())*gaussian

def gauss_sum(x, a0,  mu0, sigma0, a1, mu1, sigma1):
    return a0*stats.norm.pdf(x, mu0, sigma0)  + a1*stats.norm.pdf(x, mu1, sigma1)

def dirac(x, x0):
    return np.where((x - x0) == 0, 1, 0)


<h1> __________________________ </h1>
<h2> Global parameters </h2>

-  <p style="font-size: 17px; color: black;"> <b> Pandas dataset params: </b> Allows to set the max number of columns and rows that are shown in a pandas dataset. </p>

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 300)

-  <p style="font-size: 17px; color: black;"> <b> Plots params: </b> In case you want to fix the parameters for all plots. </p>

In [7]:
# plt.rcParams["figure.figsize"] = 10, 8
# plt.rcParams["font.size"] = 10

-  <p style="font-size: 17px; color: black;"> <b> Analysis parameters </b> </p>

 -  <p style="font-size: 17px; color: black;"> <b> patata: </b> patata. </p>
 


In [8]:
dt = 1e3 # [ns] = 1us


<h1> __________________________ </h1>
<h2> Data </h2>

-  <p style="font-size: 17px; color: black;"> <b> Reading the file </b> </p>

-  <p style="font-size: 17px; color: black;"> File's path </p>

In [9]:
path = '/home/investigator/mariandbt/nexus/'

filename = os.path.join(path, "FibBarrMeth.next.h5")

-  <p style="font-size: 17px; color: black;"> In a .h5 file there are several objects, so we print them all to then pick the one we're interested in </p>

In [10]:
with tb.open_file(filename) as file:
    print(file)

/home/investigator/mariandbt/nexus/FibBarrMeth.next.h5 (File) ''
Last modif.: '2023-07-20T10:11:38+00:00'
Object Tree: 
/ (RootGroup) ''
/MC (Group) ''
/MC/configuration (Table(40,)) ''
/MC/hits (Table(0,)) ''
/MC/particles (Table(21,)) ''
/MC/sns_positions (Table(0,)) ''
/MC/sns_response (Table(0,)) ''



-  <p style="font-size: 17px; color: black;"> Read the file and make a copy to work with, this way it's easier to re-start the copy instead of re-reading the file. </p>

In [11]:
# %%timeit # WITH THIS YOU'RE ONLY TIMING, BUT IT DOESN'T REALLY RUN THE COMMANDS YOU TIME
data = pd.read_hdf(filename, "/MC/particles")


In [12]:
dst = data.copy()

In [13]:
dst

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,final_y,final_z,final_t,initial_volume,final_volume,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
0,0,1,opticalphoton,1,0,-251.631439,-11.722586,578.089417,0.000000,-368.963715,-155.207001,750.000000,0.843250,WORLD,TEFLON2,-3.248915e-06,-3.973064e-06,4.760180e-06,-3.248915e-06,-3.973064e-06,4.760180e-06,7.000000e-06,252.800095,none,Transportation
1,1,1,opticalphoton,1,0,411.896210,-266.444427,663.686584,0.000000,429.879669,-254.659744,694.842651,0.126271,WORLD,B2-2874,3.325433e-06,2.179195e-06,5.761302e-06,6.749935e-06,-1.029605e-08,1.854258e-06,7.000000e-06,37.854855,none,OpWLS
2,2,1,opticalphoton,1,0,-264.490784,-411.679077,-184.107483,0.000000,-259.696808,-426.963318,-141.594116,0.151542,WORLD,B2-2083,7.386571e-07,-2.354993e-06,6.550450e-06,3.624314e-07,-4.218515e-06,5.574296e-06,7.000000e-06,45.431004,none,OpWLS
3,2,4,opticalphoton,0,1,-259.696808,-426.963318,-141.594116,2.551542,-342.998230,-364.489319,740.764526,107.714569,B2-2083,TEFLON_CYLINDER,-3.228449e-08,2.167144e-07,4.571813e-07,-4.658369e-07,-1.993360e-07,-1.683342e-08,5.069735e-07,30568.955078,OpWLS,Transportation
4,2,3,opticalphoton,0,1,-259.696808,-426.963318,-141.594116,2.551542,-259.705231,-427.047607,-141.509247,2.552146,B2-2083,B2,-2.158544e-07,-2.246160e-06,2.341142e-06,-6.044028e-08,-2.420129e-06,2.170737e-06,3.251580e-06,0.120192,OpWLS,OpWLS
5,2,6,opticalphoton,0,3,-259.705231,-427.047607,-141.509247,11.052147,402.560760,-297.397186,661.568726,178.123047,B2,TEFLON_CYLINDER,6.038503e-07,2.482165e-06,1.292135e-06,2.499018e-06,1.389746e-06,1.375001e-07,2.862759e-06,49899.656250,OpWLS,Transportation
6,2,5,opticalphoton,0,3,-259.705231,-427.047607,-141.509247,11.052147,343.917603,363.621948,-135.474045,98.626984,B2,TEFLON_CYLINDER,7.824590e-08,2.542274e-07,-2.845830e-07,-1.344657e-07,3.652386e-07,-1.616120e-08,3.895401e-07,25396.941406,OpWLS,Transportation
7,2,2,opticalphoton,0,1,-259.696808,-426.963318,-141.594116,2.551542,-259.697449,-426.964478,-141.591583,2.551558,B2-2083,B2-2083,-1.718658e-06,1.903417e-06,2.889561e-06,-1.855088e-06,1.368348e-06,3.100422e-06,3.863463e-06,0.003049,OpWLS,OpWLS
8,2,7,opticalphoton,0,2,-259.697449,-426.964478,-141.591583,4.951558,137.447083,214.928482,750.000000,111.029900,B2-2083,TEFLON2,1.803340e-06,1.615342e-06,-1.009579e-06,7.742012e-07,-1.815106e-06,1.728183e-06,2.623092e-06,31675.037109,OpWLS,Transportation
9,3,1,opticalphoton,1,0,32.245838,-320.447113,202.764832,0.000000,-232.911545,-441.990631,423.767761,1.220697,WORLD,B2-2113,-5.071930e-06,-2.324884e-06,4.227344e-06,-5.955380e-06,-3.615279e-06,-6.805865e-07,7.000000e-06,365.955688,none,OpWLS


In [14]:
np.shape(dst), len(dst)

((21, 25), 21)

In [15]:
len(dst.loc[dst['final_volume'] == 'F_SENSOR_SENSAREA'])

0

In [16]:
print(len(dst.loc[dst['final_volume'] == 'F_SENSOR_SENSAREA']))
print(len(dst.loc[(dst['final_volume'].str.contains('SENSAREA')) & (dst['initial_volume'].str.contains('B2'))]))

0
0


In [17]:
dst.loc[(dst['final_volume'].str.contains('SENSOR')) & (dst['initial_volume'].str.contains('B2'))]

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,final_y,final_z,final_t,initial_volume,final_volume,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc


In [18]:
dst['length'].mean()

7505.36328125

In [19]:
print(len(dst.loc[dst['primary'] == 1]))
print(len(dst.loc[(dst['primary'] == 1) & (dst['initial_volume'].str.contains('WORLD'))]))

10
10
